<a href="https://colab.research.google.com/github/agar1162/Microxcoding/blob/main/v2_Microxcoding_VIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/microsoft/microxcaling.git
!pip install torch==2.2.0 torchaudio==2.2.0
!pip install "numpy<2" --upgrade

%cd microxcaling
!pip install -e .

Cloning into 'microxcaling'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 274 (delta 37), reused 27 (delta 27), pack-reused 221 (from 1)
Receiving objects: 100% (274/274), 963.57 KiB | 48.18 MiB/s, done.
Resolving deltas: 100% (164/164), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.4/755.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━

/content/microxcaling
Obtaining file:///content/microxcaling
  Installing build dependencies ... canceled
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
^C


In [7]:
%cd /content/microxcaling
!pip install -e .

/content/microxcaling
Obtaining file:///content/microxcaling
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
  Using cached torchvision-0.25.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (5.4 kB)
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB

In [9]:
import torch
import torch.nn as nn

def make_quant_sim(model, w_elem_format, a_elem_format, block_size=32,
                   scale_bits=8, bfloat=16, custom_cuda=False,
                   quantize_backprop=False, round="nearest"):
    """
    Apply microscaling (MX) quantization to the model using the microxcaling library.

    Args:
        model: The model to quantize
        w_elem_format: Weight element format (e.g., 'fp6_e3m2', 'fp8_e4m3', 'int8')
        a_elem_format: Activation element format (e.g., 'fp6_e3m2', 'fp8_e4m3', 'int8')
        block_size: MX scaling block size (default: 32 for MX-compatible formats)
        scale_bits: Number of bits for the MX shared scale (default: 8)
        bfloat: Bfloat format for vector operations (default: 16 for bfloat16)
        custom_cuda: Use custom CUDA kernels (default: False)
        quantize_backprop: Apply quantization on backward pass (default: False)
        round: Rounding mode ('nearest', 'floor', 'even') (default: 'nearest')
    """
    try:
        from mx import finalize_mx_specs
        from mx import mx_mapping
    except ImportError:
        raise ImportError("microxcaling library not found. Please install it first.")

    # Create MX specs dictionary
    mx_specs = {
        'w_elem_format': w_elem_format,
        'a_elem_format': a_elem_format,
        'block_size': block_size,
        'scale_bits': scale_bits,
        'bfloat': bfloat,
        'custom_cuda': custom_cuda,
        'quantize_backprop': quantize_backprop,
        'round': round,
    }

    # Finalize the specs (this sets defaults and validates)
    mx_specs = finalize_mx_specs(mx_specs)

    if mx_specs is None:
        print("No quantization specified, skipping MX injection")
        return

    print(f"Applying MX quantization with specs: {mx_specs}")

    # Auto-inject MX modules and functions
    # This will replace certain torch.nn.* and torch.nn.functional.*
    # modules/functions in the global namespace!
    mx_mapping.inject_pyt_ops(mx_specs)

    print("MX quantization applied successfully!")